In [21]:
import numpy as np

In [11]:
f = open('./smsspamcollection/SMSSpamCollection.txt')
classes = []
msgs = []
classnames = ['ham', 'spam']
for line in f.readlines():
    splitted = line.split('\t')
    classes.append(0 if splitted[0] == 'ham' else 1)
    msgs.append(splitted[1].translate(None, '\n').lower())

In [13]:
(classes[0:5],msgs[0:5])

([0, 0, 1, 0, 0],
 ['go until jurong point, crazy.. available only in bugis n great world la e buffet... cine there got amore wat...',
  'ok lar... joking wif u oni...',
  "free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005. text fa to 87121 to receive entry question(std txt rate)t&c's apply 08452810075over18's",
  'u dun say so early hor... u c already then say...',
  "nah i don't think he goes to usf, he lives around here though"])

In [86]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score

In [90]:
pipeline = Pipeline([('cv', CountVectorizer()), ('logistic_reg', LogisticRegression())])
model = pipeline.fit(X=msgs, y=classes)
np.random.seed(2)

In [91]:
cv = cross_val_score(pipeline, X=msgs, y=classes, cv = 10, scoring='f1')

In [89]:
print cv

[ 0.89051095  0.86764706  0.86567164  0.90510949  0.88888889  0.88059701
  0.83969466  0.86363636  0.86363636  0.91970803]


In [95]:
print cv
write('fifth.txt',"-1")

[ 0.96598639  0.89855072  0.91549296  0.95833333  0.93706294  0.91304348
  0.94444444  0.92753623  0.92198582  0.95104895]


In [62]:
def write(fname, msg, ft = None):
    f = open(fname, "w")
    if(ft is not None):
        f.write(ft%msg)
    else:
        f.write(msg)
    f.close()

In [40]:
write('first.txt', cv.mean(), '%.1lf')

In [41]:
to_test = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
    "FreeMsg: Txt: claim your reward of 3 hours talk time",
    "Have you visited the last lecture on physics?",
    "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
    "Only 99$"]

In [67]:
write('second.txt', ' '.join([str(i) for i in model.predict(to_test)]))

In [68]:
ngram_ranges = [(2,2),(3,3),(1,3)]

In [69]:
pipeline_ngrams = [Pipeline([('cv', CountVectorizer(ngram_range=i)), ('logistic_reg', LogisticRegression())]) for i in ngram_ranges]
cv_ngrams = []
for pip in pipeline_ngrams:
    cv_ngrams.append(cross_val_score(pip, X=msgs, y=classes, cv = 10, scoring='f1'))

In [73]:
cv_ngrams_means = [i.mean() for i in cv_ngrams]
print cv_ngrams_means
write('third.txt', ' '.join([('%.2lf')%i for i in cv_ngrams_means]))

[0.82242206641871329, 0.72501615554673771, 0.92513825586488374]


In [74]:
from sklearn.naive_bayes import MultinomialNB

In [79]:
Xs = [CountVectorizer(ngram_range=i).fit_transform(msgs) for i in ngram_ranges]

In [80]:
modelsNB = [MultinomialNB().fit(X,classes) for X in Xs]

In [83]:
cv_ngramsNB = []
for X in Xs:
    cv_ngramsNB.append(cross_val_score(MultinomialNB(), X=X, y=classes, cv = 10, scoring='f1'))

In [85]:
cv_ngrams_meansNB = [i.mean() for i in cv_ngramsNB]
print cv_ngrams_meansNB
write('fourth.txt', ' '.join([('%.2lf')%i for i in cv_ngrams_meansNB]))

[0.64545540135589818, 0.37862343087618666, 0.88790546088949929]
